In [ ]:
include("../load.jl")

In [ ]:
gm = minlp(true)
set_optimizer(gm, CPLEX_SILENT)
set_param(gm, :abstol, 1e-7)
set_param(gm, :ignore_accuracy, true)
globalsolve!(gm)

In [ ]:
show_trees(gm)

In [ ]:
gm.bbls[1].learners[end]
# gm.bbls[1].actuals
# gm.bbls[1].mi_constraints

In [ ]:
IAI.show_in_browser(gm.bbls[2].learners[end])

In [ ]:
bbr = gm.bbls[1]
score = IAI.score(bbr.learners[end], bbr.X, bbr.Y)

In [ ]:
using Latexify
latexify(round.(gm.solution_history, sigdigits=5); env=:table, latex=false)

In [ ]:
# Printing numbers of variables and constraints
println("Variables: $(length(JuMP.all_variables(gm.model)))")
println("Constraints: $(sum(length(all_constraints(gm.model, type[1], type[2])) for type in JuMP.list_of_constraint_types(gm.model))
)")

In [ ]:
using Plots
using LaTeXStrings
# First the constraint
bbc = gm.bbls[1]
feas_idxs = findall(x -> x .>= 0, bbc.Y)
infeas_idxs = findall(x -> x .< 0, bbc.Y)
scatter(bbc.X[feas_idxs,1],
bbc.X[feas_idxs, 2], bbc.X[feas_idxs, 3], 
        legend = false, label = "feasible", color = :green,  fmt = :png)
plt = scatter!(bbc.X[infeas_idxs,1],
bbc.X[infeas_idxs, 2], bbc.X[infeas_idxs, 3], 
        title = L"\mathrm{Samples~from~constraint}~g_1(\mathbf{x}) \geq 0",
xlabel = L"x_1", ylabel = L"x_2", zlabel = L"x_3", xticks = 0:2:2, yticks = 0:2:2, zticks = 0:1:1,
legend = true, label = "infeasible", color = :red,  fmt = :png)
display(plt)
savefig(plt, "constr")

In [ ]:
Y = IAI.predict(bbc.learners[end], bbc.X)
trues = [i for i=1:length(Y) if Y[i] == (bbc.Y[i] >= 0)];
falses = [i for i=1:length(Y) if Y[i] != (bbc.Y[i] >= 0)];
axes = [1,2,3]
scatter(Matrix(bbc.X)[trues, axes[1]], Matrix(bbc.X)[trues, axes[2]], Matrix(bbc.X)[trues, axes[3]], legend = false,
        color = :purple,  fmt = :png, label = "correct")
scatter!(Matrix(bbc.X)[falses, axes[1]], Matrix(bbc.X)[falses, axes[2]], Matrix(bbc.X)[trues, axes[3]], legend = true,
         title = "Prediction accuracy of OCT-H learner", 
    xlabel = L"x_1", ylabel = L"x_2", zlabel = L"x_3", xticks = 0:2:2, yticks = 0:2:2, zticks = 0:1:1,
         color = :yellow,  fmt = :png, label = "incorrect")

In [ ]:
bbr = gm.bbls[1]
plt = scatter(bbr.X[:,1],
bbr.X[:, 2], bbr.Y, 
        title = "Objective samples",
xlabel = L"x_1", ylabel = L"x_2", zlabel = L"f(\mathbf{x})", 
legend = false, color = :green,  fmt = :png)
savefig(plt, "obj")

In [ ]:
length(trues)/length(bbc.Y)

In [ ]:
add_tree_constraints!(gm)

In [ ]:
    types = JuMP.list_of_constraint_types(gm.model)
    init_constraints = sum(length(all_constraints(gm.model, type[1], type[2])) for type in types)

In [ ]:
plot(gm.solution_history[:, end], xlabel = "Iterations", ylabel = "Objective cost", legend = false)